# Лабораторная работа 2 (Linear Model)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table


### Regression

#### 1. Обработка данных

In [2]:
df = pd.read_csv('data/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5.0,90000
1,28,Female,Master's,Data Analyst,3.0,65000
2,45,Male,PhD,Senior Manager,15.0,150000
3,36,Female,Bachelor's,Sales Associate,7.0,60000
4,52,Male,Master's,Director,20.0,200000


In [3]:
X, y = df.drop(columns=['Salary', 'Job Title']), df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# X_train, y_train = X, y

gender_le = LabelEncoder()
el_le = LabelEncoder()

X_train['Gender'] = gender_le.fit_transform(X_train['Gender'])
X_test['Gender'] = gender_le.transform(X_test['Gender'])

X_train['Education Level'] = el_le.fit_transform(X_train['Education Level'])
X_test['Education Level'] = el_le.transform(X_test['Education Level'])

#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

In [4]:
metrics = regression_cross_validate(LinearRegression, X_train.to_numpy(), y_train.to_numpy(), n_folds=5)
display_metrics_table(*metrics)

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred = linear_model.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      | 11624.6       | 971.068       |
| MSE      |     2.412e+08 |   2.89352e+07 |
| R2       |     0.896435  |   0.0126421   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 243261993.36
Средняя абсолютная ошибка (MAE): 10956.04
Коэффициент детерминации (R^2): 0.89


Как можем увидеть значение метрики $R^2$ около 0.89, что означает что около 89% дисперсии данных объясняется моделью.

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Добавить регуляризацию

In [5]:
onehot = OneHotEncoder(sparse_output=False, drop='first')

categorical_features = ['Gender', 'Education Level']

encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)


scaler = StandardScaler()
num_features = ['Age', 'Years of Experience']
X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])

In [6]:
metrics = regression_cross_validate(ElasticNet, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, alpha=0.01, l1_ratio=0.7)
display_metrics_table(*metrics)

linear_model = ElasticNet(alpha=0.01, l1_ratio=0.7)
linear_model.fit(X_train_upd, y_train)

y_pred = linear_model.predict(X_test_upd)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |       Std Dev |
|:---------|----------------:|--------------:|
| MAE      | 11166.3         | 775.32        |
| MSE      |     2.34823e+08 |   2.87383e+07 |
| R2       |     0.899254    |   0.0120659   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 238966602.60
Средняя абсолютная ошибка (MAE): 10865.15
Коэффициент детерминации (R^2): 0.89


Можно увидеть, что в целом значения метрик улучшились, хоть и не совсем значительно. Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [7]:
class MyElasticNet:
    def __init__(self, alpha=1.0, l1_ratio=0.5, max_iter=1000, tol=1e-4):
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        X_train_with_bias = np.c_[np.ones(X.shape[0]), X]
        self.coef_ = np.linalg.inv(X_train_with_bias.T @ X_train_with_bias) @ X_train_with_bias.T @ y

        l1_penalty = self.alpha * self.l1_ratio * np.sign(self.coef_)
        l2_penalty = self.alpha * (1 - self.l1_ratio) * self.coef_

        self.coef_ -= l1_penalty + l2_penalty


    def predict(self, X):
        X = np.array(X)
        X_test_with_bias = np.c_[np.ones(X.shape[0]), X]

        return X_test_with_bias @ self.coef_



In [8]:
metrics = regression_cross_validate(MyElasticNet, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, alpha=0, l1_ratio=0)
display_metrics_table(*metrics)

linear_model = MyElasticNet(alpha=0, l1_ratio=0)
linear_model.fit(X_train, y_train)

y_pred = linear_model.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      | 11624.6       | 971.068       |
| MSE      |     2.412e+08 |   2.89352e+07 |
| R2       |     0.896435  |   0.0126421   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 243261993.36
Средняя абсолютная ошибка (MAE): 10956.04
Коэффициент детерминации (R^2): 0.89


In [9]:
metrics = regression_cross_validate(MyElasticNet, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, alpha=0.01, l1_ratio=0.7)
display_metrics_table(*metrics)

linear_model = MyElasticNet(alpha=0.01, l1_ratio=0.7)
linear_model.fit(X_train_upd, y_train)

y_pred = linear_model.predict(X_test_upd)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |       Std Dev |
|:---------|----------------:|--------------:|
| MAE      | 11214.5         | 779.008       |
| MSE      |     2.35034e+08 |   2.94154e+07 |
| R2       |     0.899061    |   0.0133104   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 241968124.33
Средняя абсолютная ошибка (MAE): 10953.61
Коэффициент детерминации (R^2): 0.89


Результаты показывают что собственная имплементация модели в среднем работает на том же уровне качества, что и модель из `sklearn`

### Classification

#### 1. Обработка данных

In [10]:
df = pd.read_csv('data/Student_Depression_Dataset.csv')
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [11]:
X, y = df.drop(columns=['Depression', 'id']), df['Depression']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

categorical_features = ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
num_features = ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress', ]

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

X_train[categorical_features] = le.fit_transform(X_train[categorical_features])
X_test[categorical_features] = le.transform(X_test[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

#### 2. Построение бейзлайна 

In [12]:
metrics = classification_cross_validate(LogisticRegression, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, max_iter=1000)
display_metrics_classification_table(*metrics)

linear_model = LogisticRegression(max_iter=1000)  # Выбираем количество соседей
linear_model.fit(X_train, y_train)

y_pred = linear_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.847981 | 0.00621524 |
| Precision | 0.847526 | 0.00632844 |
| Recall    | 0.847981 | 0.00621524 |
| F1-score  | 0.847349 | 0.0062273  |

=== Результаты на Тесте ===
1. Accuracy: 83.86%
2. Precision: 83.81%
3. Recall: 83.86%
4. F1-score: 83.82%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Добавить регуляризацию

In [13]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)


scaler = StandardScaler()
X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])

/home/constantfear/.pyenv/versions/3.9.19/envs/labs/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [14]:
metrics = classification_cross_validate(LogisticRegression, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, C=0.1, max_iter=1000)
display_metrics_classification_table(*metrics)

linear_model = LogisticRegression(C=0.1, max_iter=1000)  
linear_model.fit(X_train_upd, y_train)

y_pred = linear_model.predict(X_test_upd)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.850848 | 0.00602302 |
| Precision | 0.850436 | 0.00615489 |
| Recall    | 0.850848 | 0.00602302 |
| F1-score  | 0.850196 | 0.0060759  |

=== Результаты на Тесте ===
1. Accuracy: 83.99%
2. Precision: 83.94%
3. Recall: 83.99%
4. F1-score: 83.94%


Можно увидеть что в среднем значения метрик улучшились. 
Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [15]:
class MyLogisticRegression:
    def __init__(self, C=0, learning_rate=0.01, n_iterations=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(linear_model)
            if self.C != 0:
                dw = (1 / n_samples) * np.dot(X.T, (predictions - y)) + (1 / (self.C * n_samples)) * self.weights
            else:
                dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(linear_model)
        return np.where(predictions >= 0.5, 1, 0)

In [ ]:
metrics = classification_cross_validate(MyLogisticRegression, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_iterations=1000)
display_metrics_classification_table(*metrics)

linear_model = MyLogisticRegression(n_iterations=1000)
linear_model.fit(X_train, y_train)

y_pred = linear_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.778829 | 0.00819507 |
| Precision | 0.808734 | 0.0085782  |
| Recall    | 0.778829 | 0.00819507 |
| F1-score  | 0.780033 | 0.0081312  |

=== Результаты на Тесте ===
1. Accuracy: 76.72%
2. Precision: 79.77%
3. Recall: 76.72%
4. F1-score: 76.81%


In [17]:
metrics = classification_cross_validate(MyLogisticRegression, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, C=0, n_iterations=1000)
display_metrics_classification_table(*metrics)

linear_model = MyLogisticRegression(C=0.1, n_iterations=1000)  
linear_model.fit(X_train_upd, y_train)

y_pred = linear_model.predict(X_test_upd)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.831493 | 0.00640525 |
| Precision | 0.832139 | 0.006106   |
| Recall    | 0.831493 | 0.00640525 |
| F1-score  | 0.829389 | 0.00678642 |

=== Результаты на Тесте ===
1. Accuracy: 82.07%
2. Precision: 82.08%
3. Recall: 82.07%
4. F1-score: 81.88%


Результаты показывают что собственная имплементация модели оказалась немного хуже по качеству, чем модель из `sklearn`, но с ростом числа итераций метрики сближаются (но при этом растет время обучение модели)

### Заключение

Внесённые изменения, включая нормализацию данных и добавление регуляризации в алгоритм, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 2.43e+08  | 10956.04   |  0.89      |
| Sklearn (после улучшения) | 2.38e+08  | 10865.15   |  0.89      |
| Собственная имплементация (до улучшения)   | 2.43e+08   | 10956.04    |  0.89      |
| Собственная имплементация (после улучшения)| 2.419e+08  | 10953.61    |  0.89      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-score |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   83.86%  |   83.81%   |  83.86%    |  83.82%    |
| Sklearn (после улучшения) |   83.99%  |   83.94%   |  83.99%    |  83.94%    |
| Собственная имплементация (до улучшения)   |   76.72%  |   76.77%   |  76.72%    |  76.81%    |
| Собственная имплементация (после улучшения)|   82.07%  |   82.08%   |  82.07%    |  81.88%    |